# Profitable App Profiles for the App Store and Google Play Markets

### Analyse mobile app data
#### - *The project is about....*
#### - *The goal of project is....*

### Open and explore data

In [5]:
from csv import reader
# Open and read data
def read_data(file_name, header=True):
    open_file = open(file_name, encoding='utf8')
    read_file = reader(open_file)
    dataset = list(read_file)
    if header: # The header is extracted
        return dataset[0], dataset[1:]
    else:
        return dataset

def explore_data(dataset, start, end, rows_and_columns=False):
    data_slice = dataset[start:end]
    for row in data_slice:
        print(row)
    if rows_and_columns:
        print("Summary: ")
        print("Number of rows: ", len(dataset))
        print("Number of columns: ", len(dataset[0]))

android = read_data("googleplaystore.csv") # read the googleplaystore.csv
android_header, android_file = android[0], android[1]
print(android_header)
explore_data(android_file, 0, 4, True) # 3 first rows of android_file
print("-" * 100)
ios = read_data("AppleStore.csv") # read the AppleStore.csv
ios_header, ios_file = ios[0], ios[1]
print(ios_header)
explore_data(ios_file, 0, 6, True); # 6 first rows of android_file

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
Summary: 
Number of rows:  10841
Number of columns:  13
-----------------------------------------------------------------

### Delete Wrong Data (Data cleaning)

In [6]:
# google play csv file: row 10472 incorrect rating = 19 while maximum rating is 5. This row should be deleted
print(android_header)
print(android_file[10472])
print(len(android_file))
# del android_file[10472] # must run one time
print(len(android_file))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10841
10840


### Remove duplicate value

In [7]:
# Count how many duplicate value + unique value
def duplicate_unique(file):
    duplicate = []
    unique = []
    for app in file:
        name = app[0]
        if name in unique:
            duplicate.append(name)
        else:
            unique.append(name)
    return duplicate, unique

print('Number of duplicate apps:', len(duplicate_unique(android_file)[0]))
print('\n')
print('Examples of duplicate apps:', duplicate_unique(android_file)[0][:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [8]:
# Create new dictionary {key=unique_app, value=highest_number_of_rating} => extract unique value with the highest number of rating
reviews_max = {}
for app in android_file:
    name = app[0] # current app
    n_reviews = float(app[3]) # current review
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews # update to current reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print("Actual length: ", len(reviews_max))
print("Expected length: ", len(android_file) - 1181)

Actual length:  9659
Expected length:  9659


In [9]:
# Remove duplicate value on Android dataset by using reviews_max dictionary above => create new dataset from android dataset with no duplicated value and highest number of reviews
android_clean = [] # The list to contain android clean data
already_add = [] # The name of application has been added to android_clean list above

for app in android_file:
    name = app[0]
    n_reviews = float(app[android_header.index('Reviews')])
    if reviews_max[name] == n_reviews and (name not in already_add):
        android_clean.append(app)
        already_add.append(name)

print(len(android_clean))
explore_data(android_clean, 0, 3, True)

9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
Summary: 
Number of rows:  9659
Number of columns:  13


### Removing non-english app

In [10]:
# Check the app name
# Check the app is English or not
# Check non-English character in the name app by ASCII range (0-127)

def is_english(string):
    for char in string:
        if ord(char) > 127:
            return False
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


In [11]:
# However, some english app name contain extra icon or symbol, so applying the above method might accidentally remove English app and lead to data loss
# Therefore we only defined a non-English app if its name contains more than 3 non-English character
def is_english(string):
    non_ascii = 0
    for char in string:
        if ord(char) > 127:
            non_ascii += 1
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

# Now these apps are still remained in english app list

True
True


In [12]:
# Filter all english applications in both Google Play and Apple datasets
android_english = []
ios_english = []

for app in android_clean: # Remember to use android_clean file
    name = app[0]
    if is_english(name):
        android_english.append(app)
explore_data(android_english, 0, 3, True)

for app in ios_file:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
Summary: 
Number of rows:  9614
Number of columns:  13
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

### Isolating free app:


In [14]:
# Filter all purchased app out of list. Finish data cleaning
android_final = []
ios_final = []

for app in android_english:
    price = app[android_header.index('Price')]
    if price == '0':
        android_final.append(app)

for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)

print("Length of actual android apps list: ", len(android_final))
print("Length of actual ios apps list: ", len(ios_final))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
Length of actual android apps list:  8864
Length of actual ios apps list:  3222


### Most common apps by genre
- The revenue might increase by the number of users using our apps, so it's reasonable to figure out kinds of app that attract most users

In [62]:
# Build a frequency table to show app genres on dataset in descending order -> find common app genres on dataset
# For android dataset: use 'Category' or 'Genres' column
# For ios dataset: use prime_genre column
def freq_table(dataset, index):
    table = {}
    table_percentage = {}
    total = 0

    for row in dataset:
        total += 1
        key = row[index]
        if key in table:
            table[key] += 1
        else:
            table[key] = 1

    for key in table:
        table_percentage[key] = (table[key] / total * 100)
    return table_percentage

# Sort the data in descending order, because dataset
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        pair_tuple =  (table[key], key)
        table_display.append(pair_tuple)

    table_sorted = sorted(table_display, reverse=True)
    for i in table_sorted:
        print(i[1], " : ", i[0])


print('DISPLAY')
# print(display_sorted_table(android_final, android_header.index('Genres')))
print('----------------------------------------------------')
print(display_table(android_final, android_header.index('Category')))
print('----------------------------------------------------')
print(display_table(ios_final, ios_header.index('prime_genre')))

DISPLAY
----------------------------------------------------
FAMILY  :  18.907942238267147
GAME  :  9.724729241877256
TOOLS  :  8.461191335740072
BUSINESS  :  4.591606498194946
LIFESTYLE  :  3.9034296028880866
PRODUCTIVITY  :  3.892148014440433
FINANCE  :  3.7003610108303246
MEDICAL  :  3.531137184115524
SPORTS  :  3.395758122743682
PERSONALIZATION  :  3.3167870036101084
COMMUNICATION  :  3.2378158844765346
HEALTH_AND_FITNESS  :  3.0798736462093865
PHOTOGRAPHY  :  2.944494584837545
NEWS_AND_MAGAZINES  :  2.7978339350180503
SOCIAL  :  2.6624548736462095
TRAVEL_AND_LOCAL  :  2.33528880866426
SHOPPING  :  2.2450361010830324
BOOKS_AND_REFERENCE  :  2.1435018050541514
DATING  :  1.861462093862816
VIDEO_PLAYERS  :  1.7937725631768955
MAPS_AND_NAVIGATION  :  1.3989169675090252
FOOD_AND_DRINK  :  1.2409747292418771
EDUCATION  :  1.1620036101083033
ENTERTAINMENT  :  0.9589350180505415
LIBRARIES_AND_DEMO  :  0.9363718411552346
AUTO_AND_VEHICLES  :  0.9250902527075812
HOUSE_AND_HOME  :  0.8235559

### Analyse frequency table

Analyze the frequency table you generated for the prime_genre column of the App Store dataset:
- What is the most common genre? What is the next most common?
    - Game (58.1%), entertainment(7.8%), photo and videos (4.9%)
    - Only 3.6% app genre is education followed by social networking is 3.2%
- What other patterns do you see?
    - The percentage of game is extremely higher compared to other categories
- What is the general impression — are most of the apps designed for practical purposes (education, shopping, utilities, productivity, lifestyle) or more for - entertainment (games, photo and video, social networking, sports, music)?
    -  Most of the apps on Apps Store are designed for entertainment purpose.
- Can you recommend an app profile for the App Store market based on this frequency table alone? If there's a large number of apps for a particular genre, does that - also imply that apps of that genre generally have a large number of users?
    -  It is hard and recommend new application based on this frequency table alone. In fact, a genre has large number of apps does not imply an app of that genre are well-known by the users, because it depends on the actual demand of users. We need to make further analysis to make a conclusion

Analyze the frequency table you generated for the Category and Genres column of the Google Play dataset.
- What are the most common genres?
    - Category column: FAMILY (18.9%), GAME (9.7%), TOOLS (8.4%), BUSINESS (4.6%)
    - Genre column: Tools (8.4%), Entertainment (6.0%), Education (5.3%), business (4.6%)
- What other patterns do you see?
    - Most apps in FAMILY category is actually games for kids in real Google Play Store
    - The content of both category and genre column are quite similar. However, there are many categories in the genre column that could be hard to follow for further analysis, so we'll choose 'Category' column in Android dataset.
- Compare the patterns you see for the Google Play market with those you saw for the App Store market.
    - Most apps in Apps Store are designed for entertainment purpose, while there is a balance between practical and entertainment app in the Google Play Store
- Can you recommend an app profile based on what you found so far? Do the frequency tables you generated reveal the most frequent app genres or what genres have the most users?
    - The answer is no. Since the most frequent app genre display the highest quantity of apps in a genre are produced, not which genre has the most users installed its apps.

### Most popular app by genre on the App Store
- Find the average ratings of apps in a genre.

In [78]:
# Calculate number of installs for each app genre
# Ios: use rating_count_tot column
# Android: use 'Installs' column
genre_ios = freq_table(ios_final, ios_header.index('prime_genre'))
# print(genre_ios)
print(ios_header)

avg_ios = {}
for genre in genre_ios:
    total_ratings = 0 # Sum of user ratings to each genre
    len_genre = 0 # Number of apps in each genre
    for app in ios_final:
        genre_app = app[ios_header.index('prime_genre')]
        if genre_app == genre:
            n_ratings = float(app[ios_header.index('rating_count_tot')])
            total_ratings += n_ratings
            len_genre += 1
    avg_n_ratings = total_ratings / len_genre
    avg_ios[genre] = avg_n_ratings


def sorted_dataset(dataset):
    table_sorted = []
    for key in dataset:
        table_tuple = (dataset[key], key)
        table_sorted.append(table_tuple)
    table_sorted = sorted(table_sorted, reverse=True)
    for i in table_sorted:
        print(i[1], ":", i[0])

# print(avg_ios)
print(sorted_dataset(avg_ios))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0
None


In [79]:
print(android_header)
genre_android = freq_table(android_final, android_header.index('Category'))
avg_android = {}
for genre in genre_android:
    total_installs = 0
    len_genre = 0
    for app in android_final:
        category_app = app[android_header.index('Category')]
        if category_app == genre:
            n_installs = app[android_header.index('Installs')]
            n_installs = n_installs.replace("+", "").replace(",", "") # remove redundant character: "+" or ","
            n_installs = float(n_installs) # convert to float number
            total_installs += n_installs
            len_genre += 1
    avg_n_installs = total_installs / len_genre
    avg_android[genre] = avg_n_installs

print(sorted_dataset(avg_android))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.830303

### Conclusion:
- Try to recommend an application: No fixed answer. This step will be done later after improving domain knowledge